In [1]:
"""
Functions and classes for working with CATH data.
"""

from itertools import chain
import logging
from typing import Callable, Collection, Dict, Literal, Tuple

import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from proteinclip.fasta_utils import read_fasta
from proteinclip.data_utils import DATA_DIR

CATH_DIR = DATA_DIR / "cath"
assert CATH_DIR.is_dir()


/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:20<00:00, 10.10s/it]
Device set to use cuda:0


In [2]:
class CathSequences:
    """Class for working with CATH sequences."""

    def __init__(
        self,
        version: Literal["S20", "S40"] = "S20",
        cath_dir: str = CATH_DIR,
        blacklisted_identifiers: Collection[str] | None = None,
    ):
        self._cath_dir = cath_dir

        fa_files = [f for f in CATH_DIR.glob("*.fa") if version in f.name]
        assert len(fa_files) == 1
        self._cath_fa = str(fa_files.pop())

        # Load the sequencesn
        # An example fasta key: cath|current|1a48A01/2-112
        self._seq_records = read_fasta(
            self._cath_fa, key_func=lambda s: s.split("|")[2].split("/")[0]
        )
        if blacklisted_identifiers:
            logging.info(f"Excluding n={len(blacklisted_identifiers)} identifiers")
            self._seq_records = {
                k: v
                for k, v in self._seq_records.items()
                if k not in set(blacklisted_identifiers)
            }

        # Read in the table that maps these to CATH domains
        table_files = list(CATH_DIR.glob("cath-domain-list-*.txt"))
        assert len(table_files) == 1
        self._cath_table_file = str(table_files.pop())
        self._cath_table = pd.read_csv(
            self._cath_table_file,
            comment="#",
            delimiter="\s+",
            low_memory=False,
            header=None,
        )
        self._cath_table.columns = [
            "domain",
            "class",
            "architecture",
            "topology",
            "homologous_superfamily",
            "s35_cluster",
            "s60_cluster",
            "s95_cluster",
            "s100_cluster",
            "s100_count",
            "domain_length",
            "resolution",
        ]
        self._cath_table = self._cath_table[
            self._cath_table["domain"].isin(self._seq_records.keys())
        ]
        self._cath_table = self._cath_table.set_index("domain")
        assert len(self._cath_table) == len(self._seq_records)

        # Create a new column that combines the four CATH classifications
        self._cath_table["cath_classification"] = self._cath_table[
            ["class", "architecture", "topology", "homologous_superfamily"]
        ].apply(lambda x: ".".join([str(s) for s in x]), axis=1)

        # Report lengths
        logging.info(
            f"Domain lengths from {np.min(self._cath_table['domain_length'])} to {np.max(self._cath_table['domain_length'])}"
        )

        # Group by homologous_superfamily
        self._grouped = self._cath_table.groupby("cath_classification").groups
        self._multi_member_groups = {
            k: v for k, v in self._grouped.items() if len(v) > 1
        }
        logging.info(
            f"Found {len(self._multi_member_groups)} multi-member groups among {len(self._grouped)} total groups."
        )

    def get_sequence(self, identifier: str) -> str:
        """Get the sequence for the identifier if it exists; empty string otherwise."""
        return self._seq_records.get(identifier, "")

    def get_sequence_items(self) -> Collection[Tuple[str, str]]:
        """Return the sequence items as a collection of (identifier, sequence) tuples."""
        return list(self._seq_records.items())

    def __len__(self) -> int:
        return len(self._seq_records)

    def build_ref_query_set(self) -> Tuple[Dict[str, str], Dict[str, str]]:
        """
        Build a dataset to test retrieval, inspired by:
        https://www.frontiersin.org/articles/10.3389/fbinf.2022.1033775/full

        The reference dataset is all domains
        The query dataset is all domains that belong to a multi-member CATH group
        Return each as a dictionary of {domain: sequence} mappings.
        """
        multi_member_seqs = set(chain.from_iterable(self._multi_member_groups.values()))
        references = {k: self._seq_records[k] for k in self._seq_records.keys()}
        queries = {k: self._seq_records[k] for k in multi_member_seqs}
        assert set(queries.keys()) < set(references.keys())
        return references, queries

    def get_cath_class(self, domain: str) -> str:
        """Return the CATH class for a domain."""
        return self._cath_table.loc[domain]["cath_classification"]

    def cath_class_is_multidomain(self, cath_class: str) -> bool:
        """Return True if the CATH class is a multi-domain class."""
        return cath_class in self._multi_member_groups



In [3]:
logging.basicConfig(level=logging.INFO)
x = CathSequences()


INFO:root:Read 14942 sequences from /storage/ice1/6/2/savunuri3/cse7850-compbio-project/proteinclip/data/cath/cath-dataset-nonredundant-S20-v4_3_0.fa
INFO:root:Domain lengths from 9 to 1202
INFO:root:Found 1543 multi-member groups among 5559 total groups.


In [4]:
x.get_sequence_items()[0]

('12asA00',
 'MKTAYIAKQRQISFVKSHFSRQLEERLGLIEVQAPILSRVGDGTQDNLSGAEKAVQVKVKALPDAQFEVVHSLAKWKRQTLGQHDFSAGEGLYTHMKALRPDEDRLSPLHSVYVDQWDWERVMGDGERQFSTLKSTVEAIWAGIKATEAAVSEEFGLAPFLPDQIHFVHSQELLSRYPDLDAKGRERAIAKDLGAVFLVGIGGKLSDGHRHDVRAPDYDDWSTPSELGHAGLNGDILVWNPVLEDAFELSSMGIRVDADTLKHQLALTGDEDRLELEWHQALLRGEMPQTIGGGIGQSRLTMLLLQLPHIGQVQAGVWPAAVRESVPSLL')

In [5]:
ref, query = x.build_ref_query_set()

In [6]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from esm import pretrained
from importlib import reload
from proteinclip.model_utils import ONNXModel

In [7]:
model_path = "pretrained/proteinclip_esm2_33.onnx"
proteinclip_model = ONNXModel(model_path)

2025-04-25 11:09:43.103485554 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2034553, index: 12, mask: {13, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-25 11:09:43.103570501 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2034579, index: 38, mask: {39, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-25 11:09:43.103460258 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2034555, index: 14, mask: {15, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-25 11:09:43.103572509 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2034580, index: 39, mask: {40, }, error code: 22 error msg: Invalid argument. Speci

In [8]:
# Load ESM2-33 model
esm_model, alphabet = pretrained.esm2_t33_650M_UR50D()
esm_model.eval()
batch_converter = alphabet.get_batch_converter()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
esm_model.to(device)


ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

In [32]:
import onnxruntime as ort

session = ort.InferenceSession("project3d_2_10000.onnx", providers=["CUDAExecutionProvider"])  # or "CPUExecutionProvider"


2025-04-25 11:44:01.862209256 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2062410, index: 0, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-25 11:44:01.862265359 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2062411, index: 1, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-25 11:44:01.862276475 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2062412, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2025-04-25 11:44:01.864140216 [E:onnxruntime:Default, env.cc:234 ThreadMain] pthread_setaffinity_np failed for thread: 2062445, index: 35, mask: {36, }, error code: 22 error msg: Invalid argument. Specify the

In [33]:
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
input_type = session.get_inputs()[0].type
print(f"Input name: {input_name}, shape: {input_shape}, type: {input_type}")


Input name: input, shape: ['batch_size', 1280], type: tensor(float)


In [34]:
def embed_batch_esm2(seqs, repr_layer=33, device="cuda"):
    batch_data = [(f"seq{i}", seq) for i, seq in enumerate(seqs)]
    _, _, batch_tokens = batch_converter(batch_data)
    batch_tokens = batch_tokens.to(device)

    with torch.no_grad():
        results = esm_model(batch_tokens, repr_layers=[repr_layer], return_contacts=False)
    
    embeddings = []
    for i in range(len(seqs)):
        # Extract per-token representations and average (skip BOS and EOS)
        token_reps = results["representations"][repr_layer][i, 1:-1]
        avg_emb = token_reps.mean(0).cpu().numpy()
        embeddings.append(avg_emb)

    return embeddings

def compute_proteinclip_embeddings(ls, col="sequence", batch_size=32, device="cuda"):
    clip_embeds = []

    for i in tqdm(range(0, len(ls), batch_size)):
        batch_seqs = ls[i:i+batch_size]

        # Step 1: ESM2 embeddings — returns (batch_size, emb_dim)
        esm_embs = embed_batch_esm2(batch_seqs, device=device)  # numpy or torch tensor

        # Step 2: Normalize the batch
        norms = np.linalg.norm(esm_embs, axis=1, keepdims=True)
        norms[norms == 0] = 1e-8  # prevent division by zero
        esm_embs = esm_embs / norms

        # Step 3: Batch prediction with ProteinCLIP model
        batch_clip_embs = proteinclip_model.predict_batch(esm_embs)  # shape: (B, 128)

        clip_embeds.append(batch_clip_embs)

    return np.concatenate(clip_embeds, axis=0)  # shape: (N, 128)

def compute_proteinclip_plus_embeddings(ls, col="sequence", batch_size=32, device="cuda"):
    clip_embeds = []

    for i in tqdm(range(0, len(ls), batch_size)):
        batch_seqs = ls[i:i+batch_size]

        # Step 1: ESM2 embeddings — returns (batch_size, emb_dim)
        esm_embs = embed_batch_esm2(batch_seqs, device=device)  # numpy or torch tensor

        # Step 2: Normalize the batch
        esm_embs = esm_embs / np.linalg.norm(esm_embs, axis=1, keepdims=True)  # shape: (B, D)

        # Step 3: Batch prediction with ProteinCLIP model
        batch_clip_embs = proteinclip_model.predict_batch(esm_embs)  # shape: (B, 128)
        
        batch_clip_embs = np.asarray(batch_clip_embs, dtype=np.float32)


        batch_clip_plus_embs = session.run(None, {input_name: batch_clip_embs})[0]  



        clip_embeds.append(batch_clip_plus_embs)

    return np.concatenate(clip_embeds, axis=0)  # shape: (N, 128)


def compute_proteinclip_plus_plus_embeddings(ls, col="sequence", batch_size=32, device="cuda"):
    clip_embeds = []

    for i in tqdm(range(0, len(ls), batch_size)):
        batch_seqs = ls[i:i+batch_size]

        # Step 1: ESM2 embeddings — returns (batch_size, emb_dim)
        esm_embs = embed_batch_esm2(batch_seqs, device=device)  # numpy or torch tensor

        # Step 2: Normalize the batch
        norms = np.linalg.norm(esm_embs, axis=1, keepdims=True)
        norms[norms == 0] = 1e-8  # prevent division by zero
        esm_embs = esm_embs / norms


        batch_clip_plus_plus_embs = session.run(None, {input_name: esm_embs})[0]  


        clip_embeds.append(batch_clip_plus_plus_embs)

    return np.concatenate(clip_embeds, axis=0)  # shape: (N, 128)


def compute_proteinclip_trained_embeddings(ls, col="sequence", batch_size=32, device="cuda"):
    clip_embeds = []

    for i in tqdm(range(0, len(ls), batch_size)):
        batch_seqs = ls[i:i+batch_size]

        # Step 1: ESM2 embeddings — returns (batch_size, emb_dim)
        esm_embs = embed_batch_esm2(batch_seqs, device=device)  # numpy or torch tensor

        # Step 2: Normalize the batch
        esm_embs = esm_embs / np.linalg.norm(esm_embs, axis=1, keepdims=True)  # shape: (B, D)


        batch_clip_embs = session.run(None, {input_name: esm_embs})[0]  


        clip_embeds.append(batch_clip_embs)

    return np.concatenate(clip_embeds, axis=0)  # shape: (N, 128)



In [35]:
def embed_helper(mp, embed_func):
    ref_arr = []
    ref_keys = []
    for i in mp:    
        ref_arr.append(mp[i])
        ref_keys.append(i)
        
    tqdm.write(str(max([len(i) for i in ref_arr])))
    ref_embeddings = embed_func(ref_arr, col="sequence", batch_size=128, device="cuda")
    ans = {}
    for i,key in enumerate(ref_keys):
        ans[key] = ref_embeddings[i]
    return ans

def func(cath_sequences):
    reference_sequences, query_sequences = cath_sequences.build_ref_query_set()
    ref_embeddings = embed_helper(reference_sequences, compute_proteinclip_plus_plus_embeddings)
    query_embeddings = embed_helper(query_sequences, compute_proteinclip_plus_plus_embeddings)
    return ref_embeddings, query_embeddings

    


In [36]:
def eval_embedding_function_on_dataset(
    cath_sequences: CathSequences,
    func_on_identifier: bool = True,
) -> Dict[str, Dict[str, float | str | bool]]:
    """
    Evaluate an embedding function on a dataset of sequences.
    Return a dictionary of {seq_id: {seq_id: score}} mappings.
    """
    reference_sequences, query_sequences = cath_sequences.build_ref_query_set()

    ref_embeddings, query_embeddings = func(cath_sequences)

    retval = {}
    for query, query_embeddings in tqdm(query_embeddings.items()):
        # Create a version of the reference embeddings without the query
        ref_keys = sorted(set(ref_embeddings.keys()) - {query})
        ref_embeddings_minus_query = np.array([ref_embeddings[k] for k in ref_keys])

        # Compute cosine similarity
        scores = np.dot(query_embeddings, ref_embeddings_minus_query.T)
        assert scores.ndim == 1
        max_idx = np.argmax(scores)
        matched_key = ref_keys[max_idx]

        # Get the class for both the query and the match
        query_class = cath_sequences.get_cath_class(query)
        match_class = cath_sequences.get_cath_class(matched_key)
        retval[query] = {
            "max_score": np.max(scores),
            "query_cath_class": query_class,
            "match_cath_class": match_class,
            "correct": query_class == match_class,
            "query_length": len(query_sequences[query]),
        }
    return retval


In [37]:
eval_results = eval_embedding_function_on_dataset(x)
eval_accuracy = np.array([v["correct"] for v in eval_results.values()])
print(f"Accuracy: {eval_accuracy.mean()}")

1204


100%|██████████| 117/117 [05:37<00:00,  2.88s/it]


1174


100%|██████████| 10926/10926 [01:27<00:00, 124.96it/s]


Accuracy: 0.6723412044664104


In [27]:
import pickle

In [38]:
with open('homology_pc++_10000.pkl','wb') as f:
    pickle.dump(eval_results, f)

In [62]:
import torch
import gc

torch.cuda.empty_cache()       # Frees cached memory (does NOT free tensors still referenced)
gc.collect()                   # Forces garbage collection of unreferenced objects


9087

In [63]:
for obj in dir():
    if 'tensor' in str(type(eval(obj))):
        del globals()[obj]
gc.collect()
torch.cuda.empty_cache()
